In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt

players = pd.read_csv('core/Master.csv')
salaries = pd.read_csv('core/Salaries.csv')
salaries.info()

In [ ]:
salaries_by_year = salaries.groupby(['yearID'])

## Questions to Answer

1) How has the salary for baseball been over the history of the MLB? Total? Average?
2) Whats the correlation between performance and salary?
3) What positions garner the highest salaries?
4) How is the wage disparity for baseball players? Top heavy? Even?

## What Needs to be Done

1) Need to convert the salaries dataframe to a timeindex dataframe. This will let us do a plot of salary changes over time.
2) Adjust salaries to inflation.
3) 

In [ ]:
salaries_by_year.plot()